In [4]:
import pandas as pd
import os
import numpy as np

### Importing the data

In [5]:
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
test_file_path = os.path.join(processed_data_path, 'test.csv')

In [6]:
train_df = pd.read_csv(train_file_path, index_col='PassengerId')
test_df = pd.read_csv(test_file_path, index_col='PassengerId')

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
Survived              891 non-null int64
Age                   891 non-null float64
Fare                  891 non-null float64
FamilySize            891 non-null int64
IsMother              891 non-null int64
IsMale                891 non-null int64
Deck_A                891 non-null int64
Deck_B                891 non-null int64
Deck_C                891 non-null int64
Deck_D                891 non-null int64
Deck_E                891 non-null int64
Deck_F                891 non-null int64
Deck_G                891 non-null int64
Deck_Z                891 non-null int64
Pclass_1              891 non-null int64
Pclass_2              891 non-null int64
Pclass_3              891 non-null int64
Title_Lady            891 non-null int64
Title_Master          891 non-null int64
Title_Miss            891 non-null int64
Title_Mr              891 non-null int64
Title_Mrs             891 non-

In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
Age                   418 non-null float64
Fare                  418 non-null float64
FamilySize            418 non-null int64
IsMother              418 non-null int64
IsMale                418 non-null int64
Deck_A                418 non-null int64
Deck_B                418 non-null int64
Deck_C                418 non-null int64
Deck_D                418 non-null int64
Deck_E                418 non-null int64
Deck_F                418 non-null int64
Deck_G                418 non-null int64
Deck_Z                418 non-null int64
Pclass_1              418 non-null int64
Pclass_2              418 non-null int64
Pclass_3              418 non-null int64
Title_Lady            418 non-null int64
Title_Master          418 non-null int64
Title_Miss            418 non-null int64
Title_Mr              418 non-null int64
Title_Mrs             418 non-null int64
Title_Officer         418 n

#### Data Preparation

In [9]:
X = train_df.loc[:'Age':].as_matrix().astype('float')
y = train_df['Survived'].ravel() # the numpy function is a function that converts array to one dimension.

In [10]:
print X.shape, y.shape

(891, 33) (891,)


In [11]:
#train test split
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [13]:
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(712, 33) (712,)
(179, 33) (179,)


In [14]:
# average survival in train set
print 'mean survival in train: {}'.format(np.mean(y_train))
print 'mean survival in test: {}'.format(np.mean(y_test))

mean survival in train: 0.383426966292
mean survival in test: 0.385474860335


#### Check scikit learn version

In [15]:
import sklearn

In [16]:
sklearn.__version__

'0.19.1'

### Baseline Model

In [17]:
#import function
from sklearn.dummy import DummyClassifier

In [18]:
model_dummy = DummyClassifier(strategy='most_frequent', random_state = 0)

In [19]:
#train model
model_dummy.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [20]:
print 'score for baseline model: {}'.format(model_dummy.score(X_test, y_test))

score for baseline model: 0.614525139665


In [21]:
#perfomance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [22]:
#accuracy score
print ' accuracy for the baseline model: {}'.format(accuracy_score(y_test, model_dummy.predict(X_test))) #Accuracy.

 accuracy for the baseline model: 0.614525139665


In [23]:
#confusion matrix
print ' accuracy for the baseline model: {}'.format(confusion_matrix(y_test, model_dummy.predict(X_test)))

 accuracy for the baseline model: [[110   0]
 [ 69   0]]


In [24]:
#precision and recall scores...
print 'precision for the baseline model: {}'.format(precision_score(y_test, model_dummy.predict(X_test)))
print 'recall for the baseline model: {}'.format(recall_score(y_test, model_dummy.predict(X_test)))

precision for the baseline model: 0.0
recall for the baseline model: 0.0


/Users/angiemutava/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### Kaggle trial

In [25]:
test_X = test_df.as_matrix().astype('float')

In [26]:
# get predictions
predictions = model_dummy.predict(test_X)

In [27]:
df_submission = pd.DataFrame({'PassengerId':test_df.index, 'Survived':predictions})

In [28]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [29]:
submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
submission_file_path = os.path.join(submission_data_path ,'01_dummy.csv')

In [30]:
df_submission.to_csv(submission_file_path, index=False)

In [31]:
def get_submission_file(model, filename):
    #converting to matrix.
    test_X = test_df.as_matrix().astype('float')
    #making predictions
    predictions = model.predict(test_X)
    #submission dataframe
    df_submission = pd.DataFrame({'PassengerId':test_df.index, 'Survived':predictions})
    #submission file
    submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
    submission_file_path = os.path.join(submission_data_path, filename)
    df_submission.to_csv(submission_file_path, index=False)


In [32]:
#get submission file
get_submission_file(model_dummy, '01_dummy.csv')

#### Logistic Regression Model

In [33]:
# import function
from sklearn.linear_model import LogisticRegression

In [34]:
#create model 
model_lr_1 = LogisticRegression(random_state=0)

In [35]:
# train model
model_lr_1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [41]:
#evaluate the model
print 'score for logistic regression - version 1: {}'.format(model_lr_1.score(X_test, y_test))

score for logistic regression - version 1: 1.0


In [37]:
# perfomance metrics
# accuracy
# print 'accuracy for logistic regression version 1: {}'format(accuracy_score(y_test, model_lr_1.predict() ))

In [38]:
# model coefficients. Model weights/ parameters.
model_lr_1.coef_

array([[ 6.98361095e+00, -2.59952978e-02,  1.27283697e-03,
        -2.90529116e-01,  0.00000000e+00, -6.59941916e-01,
        -2.63877143e-03, -2.98707470e-02, -2.03213009e-01,
         7.96033288e-02,  1.89730547e-01,  2.25395640e-02,
        -6.62886278e-02, -4.32008555e-01,  7.89202703e-02,
        -6.71708289e-02, -4.53895712e-01,  1.74040547e-02,
         2.04945794e-01, -8.33346277e-02, -7.83158039e-01,
         2.83726219e-01, -2.67889169e-02, -5.49407543e-02,
        -2.67293752e-01, -1.52294684e-01, -8.44694773e-02,
         6.19116432e-02, -9.13649608e-02, -5.02375294e-02,
        -3.00543780e-01, -2.78126925e-01, -1.64019345e-01]])

In [40]:
# Kaggle submission.
get_submission_file(model_lr_1, '2_sb.csv')

ValueError: X has 32 features per sample; expecting 33

#### Highperparameter optimization using scikit-learn gridsearchcv method

In [49]:
model_lr = LogisticRegression(random_state=0)

In [50]:
from sklearn.model_selection import GridSearchCV


In [63]:
parameters ={'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty':['l2'] }

In [64]:
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)

In [65]:
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [66]:
clf.best_params_

{'C': 1.0, 'penalty': 'l2'}

In [67]:
print 'best score: {}'.format(clf.best_score_)

best score: 1.0


In [68]:
#evaluate the model
print 'score for logistic  regression: {}'.format(clf.score(X_test, y_test))

score for logistic  regression: 1.0


In [71]:
# making submission.
get_submission_file(clf, '3_sb.csv')

ValueError: X has 32 features per sample; expecting 33

### Feature Normalization and Standardization.

In [72]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### Feature Normalization

In [73]:
# feature normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [74]:
X_train_scaled[:, 0].min(), X_train_scaled[:, 0].max()

(0.0, 1.0)

In [75]:
#normalize test data.
X_train_scaled = scaler.transform(X_test)

#### Feature Standardization

In [76]:
#feature standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Create model after standardization.

In [79]:
model_lr = LogisticRegression(random_state=0)
parameters ={'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty':['l1','l2'] }
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)
clf.fit(X_train_scaled, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [80]:
clf.best_score_

1.0

In [81]:
# evaluate model
print 'score for logistic regression : {}'.format(clf.score(X_test_scaled, y_test))

score for logistic regression : 1.0


#### Model Persistence

In [82]:
# pickle library is used for data persistence.
import pickle


In [83]:
# create the file paths
model_file_path = os.path.join(os.path.pardir, 'models', 'lr_model.pkl')
scaler_file_path = os.path.join(os.path.pardir, 'models', 'lr_scaler.pkl')

In [84]:
# open the files to write to.
model_file_pickle = open(model_file_path, 'wb')
scaler_file_pickle = open(scaler_file_path, 'wb')

In [85]:
# persist the model and scaler.
pickle.dump(clf, model_file_pickle)
pickle.dump(scaler, scaler_file_pickle)

In [86]:
# close the file.
model_file_pickle.close()
scaler_file_pickle.close()

#### Load the persistent file.

In [89]:
# open files in read mode.
model_file_pickle = open(model_file_path, 'r')
scaler_file_pickle = open(scaler_file_path, 'r')
# load files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)

# close the files
model_file_pickle.close()
scaler_file_pickle.close()

In [90]:
clf_loaded

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [92]:
scaler_loaded

StandardScaler(copy=True, with_mean=True, with_std=True)

In [93]:
# transform the test data using loaded scaler object.
X_test_scaled = scaler_loaded.transform(X_test)
# calculate the score using loaded model object.
print 'score for persisted logistic regression : {}'.format(clf_loaded.score(X_test_scaled, y_test))

score for persisted logistic regression : 1.0
